In [1]:

import os
import logging
import re
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics

from collections import OrderedDict
from itertools import islice

import os
import glob
import matplotlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
# Get the PyRadiomics logger (default log-level = INFO)
logger = radiomics.logger
logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

# Write out all log entries to a file
handler = logging.FileHandler(filename='./Radiomics/radiomics_Log.txt', mode='w')
formatter = logging.Formatter('%(levelname)s:%(name)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [28]:
# Use a parameter file, this customizes the extraction settings and also specifies the input image types to use and which features should be extracted
params = os.path.join('.', 'Radiomics', 'MR_5mm.yaml')

extractor = featureextractor.RadiomicsFeatureExtractor(params)

print('Enabled input images:')
for imageType in extractor.enabledImagetypes.keys():
    print('\t' + imageType)
print('Enabled features:\n\t', extractor.enabledFeatures)
print('Extraction parameters:\n\t', extractor.settings)

Enabled input images:
	Original
	LoG
	Wavelet
Enabled features:
	 {'shape': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrlm': None, 'glszm': None, 'gldm': None}
Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'correctMask': True, 'binWidth': 25, 'voxelArrayShift': 300}


In [62]:
# Setup data directory
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = os.getcwd() if directory is None else directory
print(root_dir)



def create_dict():
    # Set the base path for the data
    base_path = os.path.join(root_dir, 'Data/Training')  
    
    # Set the path for the label folder
    label_path = os.path.join(base_path, 'Label')
    
    # Find all label files
    label_files = glob.glob(os.path.join(label_path, 'THIGH_*.nii.gz'))
    
    # Initialize an empty list for the dictionary
    ddict = []
    
    # Loop through each label file
    for label_file in label_files:
        # Extract the label file name
        label_name = os.path.basename(label_file)
        
        # Extract the image file names for each type
        t1_files = glob.glob(os.path.join(base_path, 'T1', label_name.replace('.nii.gz', '_*.nii.gz')))
        
        # Create a dictionary for each set of files
        for i in range(len(t1_files)):
            ddict.append({'image': t1_files[i], 'label': label_file})

    
    return ddict

T1_dict_other = [{'image': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_016_0002.nii.gz',
                  'label': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_016_others.nii.gz'},
                 {'image': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_036_0002.nii.gz',
                  'label': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_036_others.nii.gz'},
                 {'image': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_046_0002.nii.gz',
                  'label': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_046_others.nii.gz'},
                 {'image': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_060_0002.nii.gz',
                  'label': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_060_others.nii.gz'},
                 {'image': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_065_0002.nii.gz',
                  'label': '/home/hua/Muscle_seg/Data/Training/Others/THIGH_065_others.nii.gz'},]

T1_dict_most = create_dict()

T1_dict = T1_dict_most + T1_dict_other

print(f'Training_dict1: {(len(T1_dict))}')
T1_dict[0:5]


/home/hua/Muscle_seg
Training_dict1: 66


[{'image': '/home/hua/Muscle_seg/Data/Training/T1/THIGH_014_0002.nii.gz',
  'label': '/home/hua/Muscle_seg/Data/Training/Label/THIGH_014.nii.gz'},
 {'image': '/home/hua/Muscle_seg/Data/Training/T1/THIGH_002_0002.nii.gz',
  'label': '/home/hua/Muscle_seg/Data/Training/Label/THIGH_002.nii.gz'},
 {'image': '/home/hua/Muscle_seg/Data/Training/T1/THIGH_037_0002.nii.gz',
  'label': '/home/hua/Muscle_seg/Data/Training/Label/THIGH_037.nii.gz'},
 {'image': '/home/hua/Muscle_seg/Data/Training/T1/THIGH_061_0002.nii.gz',
  'label': '/home/hua/Muscle_seg/Data/Training/Label/THIGH_061.nii.gz'},
 {'image': '/home/hua/Muscle_seg/Data/Training/T1/THIGH_027_0002.nii.gz',
  'label': '/home/hua/Muscle_seg/Data/Training/Label/THIGH_027.nii.gz'}]

In [39]:
for patient in T1_dict:
    image = patient['image']
    mask = patient['label']
    THIGH_name = re.search("THIGH_...", mask).group(0)
    #print(THIGH_name)

In [63]:
# calculating radiomic features in batch
features = {}

for patient in T1_dict:
    image = patient['image']
    mask = patient['label']
    THIGH_name = re.search("THIGH_...", mask).group(0)
    # Tumor is in label value 6
    features[THIGH_name+"_SA"] = extractor.execute( image, mask, label=1 )
    features[THIGH_name+"_RF"] = extractor.execute( image, mask, label=2 )
    features[THIGH_name+"_VL"] = extractor.execute( image, mask, label=3 )
    features[THIGH_name+"_VI"] = extractor.execute( image, mask, label=4 )
    features[THIGH_name+"_VM"] = extractor.execute( image, mask, label=5 )
    features[THIGH_name+"_AM"] = extractor.execute( image, mask, label=6 )
    features[THIGH_name+"_GR"] = extractor.execute( image, mask, label=7 )
    features[THIGH_name+"_BL"] = extractor.execute( image, mask, label=8 )
    features[THIGH_name+"_ST"] = extractor.execute( image, mask, label=9 )
    features[THIGH_name+"_SM"] = extractor.execute( image, mask, label=10 )
    features[THIGH_name+"_BB"] = extractor.execute( image, mask, label=11 )

radiomics = pd.DataFrame(features)
radiomics.to_excel('./Radiomics/Huashan_T1_radiomics.xlsx')

Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, a

In [40]:
# test

T1_dict_test = T1_dict[0:2]
T1_dict_test

[{'image': '/home/hua/Muscle_seg/Data/Training/T1/THIGH_014_0002.nii.gz',
  'label': '/home/hua/Muscle_seg/Data/Training/Label/THIGH_014.nii.gz'},
 {'image': '/home/hua/Muscle_seg/Data/Training/T1/THIGH_002_0002.nii.gz',
  'label': '/home/hua/Muscle_seg/Data/Training/Label/THIGH_002.nii.gz'}]

In [33]:
# test: calculating radiomic features in batch
features = {}

for patient in T1_dict_test:
    image = patient['image']
    mask = patient['label']
    THIGH_name = re.search("THIGH_...", mask).group(0)
    # Tumor is in label value 6
    features[THIGH_name+"_SA"] = extractor.execute( image, mask, label=1 )
    features[THIGH_name+"_RF"] = extractor.execute( image, mask, label=2 )
    features[THIGH_name+"_VL"] = extractor.execute( image, mask, label=3 )
    features[THIGH_name+"_VI"] = extractor.execute( image, mask, label=4 )
    features[THIGH_name+"_VM"] = extractor.execute( image, mask, label=5 )
    features[THIGH_name+"_AM"] = extractor.execute( image, mask, label=6 )
    features[THIGH_name+"_GR"] = extractor.execute( image, mask, label=7 )
    features[THIGH_name+"_BL"] = extractor.execute( image, mask, label=8 )
    features[THIGH_name+"_ST"] = extractor.execute( image, mask, label=9 )
    features[THIGH_name+"_SM"] = extractor.execute( image, mask, label=10 )
    features[THIGH_name+"_BB"] = extractor.execute( image, mask, label=11 )

features

Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, attempting to correct Mask
Image/Mask geometry mismatch, a

{'THIGH_014_SA': OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.1.0'),
              ('diagnostics_Versions_Numpy', '1.23.1'),
              ('diagnostics_Versions_SimpleITK', '2.2.1'),
              ('diagnostics_Versions_PyWavelet', '1.4.1'),
              ('diagnostics_Versions_Python', '3.8.13'),
              ('diagnostics_Configuration_Settings',
               {'minimumROIDimensions': 2,
                'minimumROISize': None,
                'normalize': True,
                'normalizeScale': 100,
                'removeOutliers': None,
                'resampledPixelSpacing': None,
                'interpolator': 'sitkBSpline',
                'preCrop': False,
                'padDistance': 5,
                'distances': [1],
                'force2D': False,
                'force2Ddimension': 0,
                'resegmentRange': None,
                'label': 1,
                'additionalInfo': True,
                'correctMask': True,
                'binWidth':

In [61]:
radiomics.index[:32]

Index(['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
       'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet',
       'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings',
       'diagnostics_Configuration_EnabledImageTypes',
       'diagnostics_Image-original_Hash',
       'diagnostics_Image-original_Dimensionality',
       'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size',
       'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum',
       'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash',
       'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size',
       'diagnostics_Mask-original_BoundingBox',
       'diagnostics_Mask-original_VoxelNum',
       'diagnostics_Mask-original_VolumeNum',
       'diagnostics_Mask-original_CenterOfMassIndex',
       'diagnostics_Mask-original_CenterOfMass',
       'diagnostics_Mask-corrected_Spacing', 'diagnostics_Mask-correc

In [55]:
radiomics = pd.DataFrame(features)
radiomics.to_excel('./Radiomics/radiomics.xlsx')

,THIGH_014_SA,THIGH_014_RF,THIGH_014_VL,THIGH_014_VI,THIGH_014_VM,THIGH_014_AM,THIGH_014_GR,THIGH_014_BL,THIGH_014_ST,THIGH_014_SM,...,THIGH_002_RF,THIGH_002_VL,THIGH_002_VI,THIGH_002_VM,THIGH_002_AM,THIGH_002_GR,THIGH_002_BL,THIGH_002_ST,THIGH_002_SM,THIGH_002_BB
diagnostics_Mask-corrected_VoxelNum,11682,23856,59884,58138,46678,60979,13896,25217,27712,18778,...,16031,53114,32435,35423,41438,10901,16524,12619,20396,6691
diagnostics_Mask-corrected_VolumeNum,11,4,11,17,7,5,4,5,7,7,...,3,2,5,2,2,2,2,3,2,2
diagnostics_Mask-corrected_CenterOfMassIndex,"(253.39205615476803, 241.22590309878444, 5.446...","(251.84159121395038, 202.6240358819584, 7.8430...","(243.01816845902079, 247.9466969474317, 6.6748...","(246.87884000137603, 251.3728886442602, 5.8967...","(249.14670722824457, 239.30341916963025, 3.715...","(254.7290706636711, 288.7515046163433, 7.87959...","(258.8731289579735, 282.352115716753, 6.929404...","(254.38961811476386, 318.38478010865686, 4.762...","(264.9290920900693, 328.4260609122402, 6.79636...","(246.26562999254446, 313.2728725103845, 3.3548...",...,"(248.28083089015033, 203.94042792090326, 6.929...","(258.29453628045337, 245.44894001581503, 5.159...","(253.3728688145522, 248.08641899182982, 4.9375...","(260.75942184456426, 233.2390819524038, 2.9969...","(251.02664221246198, 280.5492301752015, 7.2363...","(255.61076965416018, 275.333455646271, 6.07347...","(258.70085935608813, 316.7921205519245, 3.8973...","(249.39709961169666, 321.7598066407798, 5.2680...","(258.0424593057462, 305.59349872524024, 3.1749...","(241.93110147959945, 282.8997160364669, 1.6960..."
diagnostics_Mask-corrected_CenterOfMass,"(-0.621486379838899, -0.21092733852643164, -15...","(-3.3455831432952436, -33.38321551287777, 46.5...","(-9.208483577725815, 5.915915634603124, 16.246...","(-5.79673237546308, 8.736903844013142, -3.9876...","(-4.421743353950205, -1.6291418422850938, -60....","(2.4920258176875336, 40.44454456329149, 47.599...","(5.7560887494647375, 34.82571885590332, 22.880...","(3.2104768670567125, 66.0186265503259, -33.404...","(12.7448359231893, 74.14998766908798, 19.46741...","(-4.028879987154738, 62.00091775765031, -70.00...",...,"(-6.658704672229135, -22.822912472932842, 13.8...","(1.9200236217199915, 9.460787070291815, -32.16...","(-1.8809348525585676, 11.628720965088235, -37....","(3.4972429649631067, -0.04397041578309313, -88...","(-2.9491939270122884, 36.94424964697723, 21.88...","(0.4713966328567949, 32.821574893363874, -8.36...","(3.6225681847419935, 65.22178301578705, -64.90...","(-3.479555382317244, 69.23837540599965, -29.24...","(2.847993771908193, 56.515785715992195, -83.69...","(-10.26384826732388, 39.17289808676475, -122.1..."
diagnostics_Mask-corrected_Mean,113.881853,107.988093,159.89896,131.782823,117.477271,135.920414,178.663961,200.816118,160.010882,170.11187,...,116.660288,198.056105,164.766327,114.92633,188.967934,158.889644,223.041109,208.205837,158.719982,133.959954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,0.230113427820957,0.19280340898544665,0.108727374579938,0.1303341244794346,0.2118817389425224,0.27438813714074667,0.07767425044196075,0.11932179861978424,0.12424251493005847,0.17869475395999143,...,0.11764971063262071,0.1891358760603511,0.062248422711844394,0.16441597139016562,0.16923320953469237,0.05423579625396823,0.16219282845753033,0.14813551548187845,0.28483539118999385,0.08995122527847098
wavelet-LLL_gldm_LowGrayLevelEmphasis,0.008626793762072794,0.005018111717599959,0.0027641004901268517,0.003102006077730399,0.003943887220264827,0.0038424208806637637,0.004546474748919577,0.0029260556009049893,0.0031470961185967082,0.005316884007851938,...,0.004344532463534369,0.005104415623059883,0.003458377961707108,0.004801710970124954,0.002848056502654271,0.0031483302287317163,0.005153327152798153,0.004384147365807168,0.007782796692969775,0.003945399299935525
wavelet-LLL_gldm_SmallDependenceEmphasis,0.2961407962175105,

In [137]:
## slice dignostic variables in the feature ordereddict
sliced = islice(features['chen_chao'].items(), 22, None)  
features['chen_chao'] = OrderedDict(sliced)
features['chen_chao']['patient'] = 'chen_chao'
patient = pd.DataFrame.from_dict(features['chen_chao'], orient='index').T.set_index('patient')
patient

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,wavelet-LLL_gldm_GrayLevelNonUniformity,wavelet-LLL_gldm_GrayLevelVariance,wavelet-LLL_gldm_HighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceEmphasis,wavelet-LLL_gldm_LargeDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,wavelet-LLL_gldm_LowGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceEmphasis,wavelet-LLL_gldm_SmallDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis
patient,,,,,,,,,,,,,,,,,,,,,
chen_chao,0.372682,0.083491,39.56411,473.871477,304.76416777966625,186.39019587004233,282.3667481205459,326.63891444548767,206654.12875764674,176.603555,...,556.8349424901352,52.492806959313675,192.97430946184198,19.052220636386533,2237.4642767189994,0.8143948977997082,0.027885206461215877,0.25200569320149635,66.70707455413884,0.003970721490807247


In [121]:
features['chen_chao']['patient'] = 'chen_chao'
patient = pd.DataFrame.from_dict(features['chen_chao'], orient='index').T.set_index('patient')
patient

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,wavelet-LLL_gldm_GrayLevelNonUniformity,wavelet-LLL_gldm_GrayLevelVariance,wavelet-LLL_gldm_HighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceEmphasis,wavelet-LLL_gldm_LargeDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,wavelet-LLL_gldm_LowGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceEmphasis,wavelet-LLL_gldm_SmallDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis
patient,,,,,,,,,,,,,,,,,,,,,
chen_chao,v3.0.1,1.20.3,2.0.2,1.1.1,3.7.10,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [3.0, 5.0]},...",5c37a387de1d1d9c668ba6f431b4745380e4eb69,3D,"(0.8202999830245972, 0.8202999830245972, 26.00...",...,556.8349424901352,52.492806959313675,192.97430946184198,19.052220636386533,2237.4642767189994,0.8143948977997082,0.027885206461215877,0.25200569320149635,66.70707455413884,0.003970721490807247


In [106]:
#features['chen_chao']['original_shape_Elongation']
featurelist = list(features['chen_chao'].items())[22:]
#pd.DataFrame(featurelist)
#pd.DataFrame.from_dict(features['chen_chao'], orient = 'index')


OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'),
             ('diagnostics_Versions_Numpy', '1.20.3'),
             ('diagnostics_Versions_SimpleITK', '2.0.2'),
             ('diagnostics_Versions_PyWavelet', '1.1.1'),
             ('diagnostics_Versions_Python', '3.7.10'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': True,
               'normalizeScale': 100,
               'removeOutliers': None,
               'resampledPixelSpacing': None,
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 11,
               'additionalInfo': True,
               'binWidth': 25,
               'voxelArrayShift': 300}),
             ('diagnostics_C

In [60]:
features['chen_chao']

OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'),
             ('diagnostics_Versions_Numpy', '1.20.3'),
             ('diagnostics_Versions_SimpleITK', '2.0.2'),
             ('diagnostics_Versions_PyWavelet', '1.1.1'),
             ('diagnostics_Versions_Python', '3.7.10'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': True,
               'normalizeScale': 100,
               'removeOutliers': None,
               'resampledPixelSpacing': None,
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 11,
               'additionalInfo': True,
               'binWidth': 25,
               'voxelArrayShift': 300}),
             ('diagnostics_C

In [20]:
print('Calculating features')
featureVector = extractor.execute(Rawdata_dict_eg['fat'], Rawdata_dict_eg['label'], label =11)

for featureName in featureVector.keys():
    print('Computed %s: %s' % (featureName, featureVector[featureName]))

Calculating features
Computed diagnostics_Versions_PyRadiomics: v3.0.1
Computed diagnostics_Versions_Numpy: 1.20.3
Computed diagnostics_Versions_SimpleITK: 2.0.2
Computed diagnostics_Versions_PyWavelet: 1.1.1
Computed diagnostics_Versions_Python: 3.7.10
Computed diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 11, 'additionalInfo': True, 'binWidth': 25, 'voxelArrayShift': 300}
Computed diagnostics_Configuration_EnabledImageTypes: {'Original': {}, 'LoG': {'sigma': [3.0, 5.0]}, 'Wavelet': {}}
Computed diagnostics_Image-original_Hash: 5c37a387de1d1d9c668ba6f431b4745380e4eb69
Computed diagnostics_Image-original_Dimensionality: 3D
Computed diagnostics_Image-original_Spacing: (0.8202999830245972, 0.820

In [ ]:
featureVector.keys()

In [ ]:
Nodules = []
for l in range(1, ccif.GetObjectCount() + 1):
  Nodules.append(extractor.execute(image, newMask, label=l))